# Content-based Filtering

Aproach:
- Table mit Features und Ratings erstellen, ohne User - auf Basis des OMDB Datasets

- sklearn-kNN mit cosine-similarity darauf anwenden

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

## This is a WIP: Decided to do omdb data cleaning first - bevore a useful model can be applied

Sources:

#### https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

#### https://www.kaggle.com/johnwill225/movie-recommendations

#### https://towardsdatascience.com/how-we-built-a-content-based-filtering-recommender-system-for-music-with-python-c6c3b1020332


In [135]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA

In [80]:
movies = pd.read_csv('../data/preprocessed/movies_id_updated.csv', sep=',')
ratings = pd.read_csv('../data/raw/ratings.csv', sep=',')
omdb = pd.read_csv('../data/raw/omdb_total.csv')

In [81]:
movies = movies.rename(columns = {'id':'movieID'})
movies.head()

,movieID,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [82]:
ratings.head()

,user_id,movieID,rating
0,1264,2363,3.5
1,213,8368,2.5
2,593,64032,3.0
3,609,54995,4.0
4,1590,5005,4.0


In [83]:
rated_movies = pd.merge(movies, ratings, how = 'outer', on = 'movieID')

In [84]:
rated_movies

,movieID,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL,user_id,rating
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1339.0,5.0
1,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,551.0,3.5
2,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,336.0,4.5
3,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1087.0,3.5
4,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1598.0,4.0
...,...,...,...,...,...,...,...,...,...,...
812941,65126,Choke,tt1024715,Choke,http://ia.media-imdb.com/images/M/MV5BMTMxMDI4...,2008.0,choke,http://content6.flixster.com/movie/10/85/09/10...,599.0,3.5
812942,65130,Revolutionary Road,tt0959337,Revolutionary Road,http://ia.media-imdb.com/images/M/MV5BMTI2MzY2...,2008.0,revolutionary_road,http://content8.flixster.com/movie/10/88/40/10...,817.0,2.5
812943,65133,Blackadder Back & Forth,tt0212579,Blackadder Back & Forth,http://ia.media-imdb.com/images/M/MV5BMjA5MjU4...,1999.0,blackadder-back-forth,http://content7.flixster.com/movie/34/10/69/34...,1059.0,4.0
812944,65133,Blackadder Back & Forth,tt0212579,Blackadder Back & Forth,http://ia.media-imdb.com/images/M/MV5BMjA5MjU4...,1999.0,blackadder-back-forth,http://content7.flixster.com/movie/34/10/69/34...,480.0,5.0


In [85]:
rated_movies = rated_movies.drop(['title','imdbPictureURL','rtID','rtPictureURL','user_id'],axis=1)

In [86]:
omdb.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,Production,Website,Response,Internet Movie Database,Rotten Tomatoes,Metacritic,totalSeasons,Season,Episode,seriesID
0,0,Toy Story,1995,G,22 Nov 1995,81 min,"Animation, Adventure, Comedy, Family, Fantasy",John Lasseter,"John Lasseter (original story by), Pete Docter...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",...,NaN,NaN,True,8.3/10,100%,95/100,NaN,NaN,NaN,NaN
1,1,Jumanji,1995,PG,15 Dec 1995,104 min,"Adventure, Comedy, Family, Fantasy",Joe Johnston,"Jonathan Hensleigh (screenplay by), Greg Taylo...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",...,NaN,NaN,True,7.0/10,54%,39/100,NaN,NaN,NaN,NaN
2,2,Grumpy Old Men,1993,PG-13,25 Dec 1993,103 min,"Comedy, Drama, Romance",Donald Petrie,Mark Steven Johnson,"Jack Lemmon, Walter Matthau, Ann-Margret, Burg...",...,NaN,NaN,True,7.0/10,63%,53/100,NaN,NaN,NaN,NaN
3,3,Waiting to Exhale,1995,R,22 Dec 1995,124 min,"Comedy, Drama, Romance",Forest Whitaker,"Terry McMillan (novel), Terry McMillan (screen...","Whitney Houston, Angela Bassett, Loretta Devin...",...,NaN,NaN,True,5.9/10,56%,NaN,NaN,NaN,NaN,NaN
4,4,Father of the Bride Part II,1995,PG,08 Dec 1995,106 min,"Comedy, Family, Romance",Charles Shyer,"Albert Hackett (screenplay ""Father's Little Di...","Steve Martin, Diane Keaton, Martin Short, Kimb...",...,NaN,NaN,True,6.0/10,48%,49/100,NaN,NaN,NaN,NaN


In [87]:
omdb = omdb.drop(['Production','Website','totalSeasons','Season','Episode','seriesID','Type','BoxOffice','DVD','Response','Poster'],axis=1)

In [88]:
omdb.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,...,Language,Country,Awards,Metascore,imdbRating,imdbVotes,imdbID,Internet Movie Database,Rotten Tomatoes,Metacritic
0,0,Toy Story,1995,G,22 Nov 1995,81 min,"Animation, Adventure, Comedy, Family, Fantasy",John Lasseter,"John Lasseter (original story by), Pete Docter...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",tt0114709,8.3/10,100%,95/100
1,1,Jumanji,1995,PG,15 Dec 1995,104 min,"Adventure, Comedy, Family, Fantasy",Joe Johnston,"Jonathan Hensleigh (screenplay by), Greg Taylo...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",...,"English, French",USA,4 wins & 11 nominations.,39.0,7.0,"294,340",tt0113497,7.0/10,54%,39/100
2,2,Grumpy Old Men,1993,PG-13,25 Dec 1993,103 min,"Comedy, Drama, Romance",Donald Petrie,Mark Steven Johnson,"Jack Lemmon, Walter Matthau, Ann-Margret, Burg...",...,English,USA,1 win & 1 nomination.,53.0,7.0,"41,401",tt0107050,7.0/10,63%,53/100
3,3,Waiting to Exhale,1995,R,22 Dec 1995,124 min,"Comedy, Drama, Romance",Forest Whitaker,"Terry McMillan (novel), Terry McMillan (screen...","Whitney Houston, Angela Bassett, Loretta Devin...",...,English,USA,9 wins & 10 nominations.,NaN,5.9,"9,222",tt0114885,5.9/10,56%,NaN
4,4,Father of the Bride Part II,1995,PG,08 Dec 1995,106 min,"Comedy, Family, Romance",Charles Shyer,"Albert Hackett (screenplay ""Father's Little Di...","Steve Martin, Diane Keaton, Martin Short, Kimb...",...,English,USA,Nominated for 1 Golden Globe. Another 1 win & ...,49.0,6.0,"33,005",tt0113041,6.0/10,48%,49/100


In [89]:
movie_data = pd.merge(rated_movies, omdb, how = 'outer', on = 'imdbID')

In [90]:
movie_data.drop(['Unnamed: 0'],axis=1)

,movieID,imdbID,spanishTitle,year,rating,Title,Year,Rated,Released,Runtime,...,Plot,Language,Country,Awards,Metascore,imdbRating,imdbVotes,Internet Movie Database,Rotten Tomatoes,Metacritic
0,1.0,tt0114709,Toy story (juguetes),1995.0,5.0,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
1,1.0,tt0114709,Toy story (juguetes),1995.0,5.0,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
2,1.0,tt0114709,Toy story (juguetes),1995.0,5.0,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
3,1.0,tt0114709,Toy story (juguetes),1995.0,3.5,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
4,1.0,tt0114709,Toy story (juguetes),1995.0,3.5,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002469,NaN,tt0109688,NaN,NaN,NaN,Ashes of Time,1994,R,17 Sep 1994,100 min,...,A broken-hearted hit man moves to the desert w...,"Mandarin, Cantonese","Hong Kong, Taiwan",9 wins & 12 nominations.,69.0,7.2,"11,911",7.2/10,NaN,69/100
1002470,NaN,tt0123221,NaN,NaN,NaN,#DUPE#,1998,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23%,NaN
1002471,NaN,tt0210234,NaN,NaN,NaN,#DUPE#,1999,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19%,NaN
1002472,NaN,tt0235679,NaN,NaN,NaN,#DUPE#,2000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21%,NaN


In [91]:
movie_data.isna().sum()

movieID                        5
imdbID                         0
spanishTitle                   5
year                         385
rating                       151
Unnamed: 0                   270
Title                        270
Year                         270
Rated                       4401
Released                     958
Runtime                      287
Genre                        282
Director                    3016
Writer                      6282
Actors                       531
Plot                         506
Language                     563
Country                      324
Awards                     48118
Metascore                  94138
imdbRating                   274
imdbVotes                   1903
Internet Movie Database      274
Rotten Tomatoes            32420
Metacritic                 94132
dtype: int64

In [96]:
# Maybe drop NaNs here
movie_data.dropna()

,movieID,imdbID,spanishTitle,year,rating,Unnamed: 0,Title,Year,Rated,Released,...,Plot,Language,Country,Awards,Metascore,imdbRating,imdbVotes,Internet Movie Database,Rotten Tomatoes,Metacritic
0,1.0,tt0114709,Toy story (juguetes),1995.0,5.0,0.0,Toy Story,1995,G,22 Nov 1995,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
1,1.0,tt0114709,Toy story (juguetes),1995.0,5.0,2872.0,Toy Story,1995,G,22 Nov 1995,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
2,1.0,tt0114709,Toy story (juguetes),1995.0,5.0,4614.0,Toy Story,1995,G,22 Nov 1995,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
3,1.0,tt0114709,Toy story (juguetes),1995.0,3.5,0.0,Toy Story,1995,G,22 Nov 1995,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
4,1.0,tt0114709,Toy story (juguetes),1995.0,3.5,2872.0,Toy Story,1995,G,22 Nov 1995,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002450,64957.0,tt0421715,El curioso caso de Benjamin Button,2008.0,3.5,10174.0,The Curious Case of Benjamin Button,2008,PG-13,25 Dec 2008,...,"Tells the story of Benjamin Button, a man who ...","English, Russian, French",USA,Won 3 Oscars. Another 81 wins & 157 nominations.,70.0,7.8,"568,717",7.8/10,71%,70/100
1002451,64957.0,tt0421715,El curioso caso de Benjamin Button,2008.0,4.0,10174.0,The Curious Case of Benjamin Button,2008,PG-13,25 Dec 2008,...,"Tells the story of Benjamin Button, a man who ...","English, Russian, French",USA,Won 3 Oscars. Another 81 wins & 157 nominations.,70.0,7.8,"568,717",7.8/10,71%,70/100
1002462,65088.0,tt0960731,Más allá de los sueños,2008.0,3.5,10186.0,Bedtime Stories,2008,PG,25 Dec 2008,...,A hotel handyman's life changes when the lavis...,English,USA,3 wins & 5 nominations.,33.0,6.0,"86,128",6.0/10,26%,33/100
1002464,65126.0,tt1024715,Choke,2008.0,3.0,10188.0,Choke,2008,R,26 Sep 2008,...,A sex-addicted con-man pays for his mother's h...,English,USA,2 wins & 7 nominations.,47.0,6.4,"31,058",6.4/10,55%,47/100


In [97]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [98]:
y = np.array(movie_data['rating'])
movie_data.drop(['rating'],axis=1)

,movieID,imdbID,spanishTitle,year,Unnamed: 0,Title,Year,Rated,Released,Runtime,...,Plot,Language,Country,Awards,Metascore,imdbRating,imdbVotes,Internet Movie Database,Rotten Tomatoes,Metacritic
0,1.0,tt0114709,Toy story (juguetes),1995.0,0.0,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
1,1.0,tt0114709,Toy story (juguetes),1995.0,2872.0,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
2,1.0,tt0114709,Toy story (juguetes),1995.0,4614.0,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
3,1.0,tt0114709,Toy story (juguetes),1995.0,0.0,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
4,1.0,tt0114709,Toy story (juguetes),1995.0,2872.0,Toy Story,1995,G,22 Nov 1995,81 min,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,95.0,8.3,"852,896",8.3/10,100%,95/100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002450,64957.0,tt0421715,El curioso caso de Benjamin Button,2008.0,10174.0,The Curious Case of Benjamin Button,2008,PG-13,25 Dec 2008,166 min,...,"Tells the story of Benjamin Button, a man who ...","English, Russian, French",USA,Won 3 Oscars. Another 81 wins & 157 nominations.,70.0,7.8,"568,717",7.8/10,71%,70/100
1002451,64957.0,tt0421715,El curioso caso de Benjamin Button,2008.0,10174.0,The Curious Case of Benjamin Button,2008,PG-13,25 Dec 2008,166 min,...,"Tells the story of Benjamin Button, a man who ...","English, Russian, French",USA,Won 3 Oscars. Another 81 wins & 157 nominations.,70.0,7.8,"568,717",7.8/10,71%,70/100
1002462,65088.0,tt0960731,Más allá de los sueños,2008.0,10186.0,Bedtime Stories,2008,PG,25 Dec 2008,99 min,...,A hotel handyman's life changes when the lavis...,English,USA,3 wins & 5 nominations.,33.0,6.0,"86,128",6.0/10,26%,33/100
1002464,65126.0,tt1024715,Choke,2008.0,10188.0,Choke,2008,R,26 Sep 2008,92 min,...,A sex-addicted con-man pays for his mother's h...,English,USA,2 wins & 7 nominations.,47.0,6.4,"31,058",6.4/10,55%,47/100


In [141]:
# Remove all '%' and convert to float
movie_data['Rotten Tomatoes'] = movie_data['Rotten Tomatoes'].str.replace(r'\D', '').astype(float)

In [142]:
movie_data['Rotten Tomatoes']

0          100.0
1          100.0
2          100.0
3          100.0
4          100.0
           ...  
1002450     71.0
1002451     71.0
1002462     26.0
1002464     55.0
1002465     55.0
Name: Rotten Tomatoes, Length: 855928, dtype: float64

In [143]:
X = np.array(movie_data)

In [144]:
le = preprocessing.LabelEncoder()
for i in range(0,23):
    X[:,i] = le.fit_transform(X[:,i])

In [145]:
y = le.fit_transform(y)

In [146]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/5)

In [147]:
print("Training")
knn.fit(X_train, y_train)

Training


ValueError: could not convert string to float: '74/100'

In [ ]:
# Do predictions
print("Do Prediction")
pred = knn.predict(X_test)

In [ ]:
# Check Accuracy
print("Accuracy : {}".format(accuracy_score(y_test, pred)))
#RMSE for ratings [0,5]
print("RMSE : {}".format(mean_squared_error(y_test, pred, squared=False)))

In [ ]:
## function that predicts the rating of a movie from its imdbID and its 20 nearest neighbors

def predict_movie_rating(imdbID):
    # calculate the average rating of each movie
    # get 20 (or x) nearest neighbors
    # compute rating of movie(imbdID) based on the rating of the 20 nearest neighbours
    # return rating prediciton
    